In [ ]:
import csv
import json
import os
import random
import sys
from time import sleep
from lxml import html
import requests

user_agent_list = ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) \
                        Gecko/20100101 Firefox/61.0',
                       'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
                        (KHTML, like Gecko) Chrome/66.0.3359.117 Safari/537.36',
                       'Mozilla/5.0 (X11; Linux x86_64; rv:61.0) \
                        Gecko/20100101 Firefox/61.0',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                        AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/60.0.3112.113 Safari/537.36',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                        AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/63.0.3239.132 Safari/537.36',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                        AppleWebKit/537.36 (KHTML, like Gecko) \
                        Chrome/66.0.3359.117 Safari/537.36',
                       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) \
                        AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 \
                        Safari/603.3.8',
                       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) \
                        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 \
                        Safari/537.36',
                       'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) \
                        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 \
                        Safari/537.36']

In [ ]:
def parse_product(url):
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)

    for i in range(20):
        sleep(random.randint(1, 3))
        try:
            doc = html.fromstring(page.content)
            XPATH_NAME = '//h1[@id="title"]//text()'
            XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P")\
                    #or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()'
            XPATH_RATING = '//i[contains(@data-hook,"average-star-rating")]/span[contains(@class, "a-icon-alt")]/text()'
            XPATH_NO_OF_RVWS= '//h2[@data-hook="total-review-count"]//text()'
            XPATH_ASIN = '//li/@data-asin//text()'
            
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            #RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAW_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            RAW_RATING = doc.xpath(XPATH_RATING)
            RAW_NO_OF_RVWS = doc.xpath(XPATH_NO_OF_RVWS)
            #RAW_ASIN = doc.xpath(XPATH_ASIN)

            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            #ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAW_AVAILABILITY).strip() if RAW_AVAILABILITY else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            REVIEWS =  ''.join(RAW_NO_OF_RVWS).strip() if RAW_NO_OF_RVWS else None
            #ASIN = ''.join(RAW_ASIN).strip() if RAW_ASIN else None
            REVIEWS = REVIEWS.split(" ")[0]
            RATING = RATING.split(" ")[0]

            if not NAME:
                raise ValueError('Captcha?')

            data = {'ASIN':url.split("/dp/")[1].split('/')[0],
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'RATING':RATING,
                    'NO_OF_REVIEWS':REVIEWS,
                   }

            return data

        except Exception as e:
            print(e)



In [ ]:
def parse_p_url_list(url):
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)
    
    data=[]
    for i in range(1):
        sleep(random.randint(1, 3))
        try:
            doc = html.fromstring(page.content)
            XPATH_NAME = '//a[@class="a-link-normal s-access-detail-page  s-color-twister-title-link a-text-normal"]/@href'
            RAW_NAME = doc.xpath(XPATH_NAME)
            NAME = [x for x in doc.xpath(XPATH_NAME) if RAW_NAME]
           
            return NAME


        except Exception as e:
            print(e)

In [ ]:
def parse_nxt_page (url):
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    page = requests.get(url, headers=headers)
    sleep(random.randint(1, 3))
    doc = html.fromstring(page.content)
    XPATH_NEXT = '//a[@class="pagnNext"]/@href'
    RAW_NEXT = doc.xpath(XPATH_NEXT)
    if len(RAW_NEXT)!=0:
        RAW_NEXT=RAW_NEXT[0]
    return 'https://www.amazon.in'+str(RAW_NEXT)

In [ ]:
parse_product('https://www.amazon.in/AmazonBasics-Apple-Certified-Lightning-Cable/dp/B010S9M3L6/ref=lp_3848980031_1_2/262-5683339-4089066?srs=3848980031&ie=UTF8&qid=1552423759&sr=8-2')

In [ ]:
page_urls=['https://www.amazon.in/l/3848980031']
i=0
product_lst=[]
while page_urls[i]!='https://www.amazon.in[]':
    product_url_list=[]
    page_urls+=[parse_nxt_page(page_urls[i])]
    product_url_list+=parse_p_url_list(page_urls[i])
    i+=1
    for product in product_url_list:
        product_lst.append(parse_product(product))
    print("  total products scrapped:  {0}      page No:  {1}".format(len(product_lst),i))
print("done")
    
    


In [ ]:
product_lst[2]

In [ ]:
import json
with open('amazonbasics.json', 'w') as outfile:
    json.dump(product_lst, outfile)

In [ ]:
extracted_data = []
for url in a[:1]:
            #url = 'http://www.amazon.in/dp/' + asin
            #url_list.append(url)
            #print('Processing: ' + url)
            extracted_data.append(parse(url))
extracted_data

In [2]:
import pandas as pd
df=pd.read_json("amznin1.json")
df.head()

,Avg_rating,Price,Product,Total_rating_no,Type/Size
0,4.7 out of 5 stars,"1,399.00","AmazonBasics Vault Case for Nintendo Switch, Red",NaN,
1,3.9 out of 5 stars,199.00,AmazonBasics Microfiber Cleaning Cloth - 222 G...,NaN,Pack of 6
2,3.9 out of 5 stars,239.00,AmazonBasics Micro USB Charging Cable for Andr...,NaN,3 Feet
3,4.0 out of 5 stars,569.00,AmazonBasics Digital Audio Coaxial Cable - 15 ...,NaN,15 Feet
4,3.9 out of 5 stars,199.00,AmazonBasics AAA Performance Alkaline Non-Rech...,NaN,AAA


In [4]:
df1=pd.read_json("amazonbasics.json")
df1.head(100)

,ASIN,AVAILABILITY,CATEGORY,NAME,NO_OF_REVIEWS,RATING,SALE_PRICE,URL
0,B00O869QUC,In stock.,Electronics > Accessories > General Purpose Ba...,AmazonBasics AAA Performance Alkaline Non-Rech...,823,3.9,199.00,https://www.amazon.in/AmazonBasics-Performance...
1,B010S9M3L6,In stock.,Computers & Accessories > Accessories & Periph...,AmazonBasics Apple Certified Lightning to USB ...,"2,000",3.6,699.00,https://www.amazon.in/AmazonBasics-Apple-Certi...
2,B013R7XPB8,In stock.,Car & Motorbike > Car & Motorbike Care > Spong...,AmazonBasics Microfiber Cleaning Cloth - 222 G...,"1,016",3.9,199.00,https://www.amazon.in/AmazonBasics-Microfiber-...
3,B01LP0UKJE,In stock.,"Sports, Fitness & Outdoors > Exercise & Fitnes...",AmazonBasics 13mm Extra Thick Yoga and Exercis...,561,3.9,949.00,https://www.amazon.in/AmazonBasics-Extra-Thick...
4,B00WRDS8H0,In stock.,Furniture > Study & Home Office Furniture,AmazonBasics Ventilated Laptop Stand (Black),"1,056",4.3,699.00,https://www.amazon.in/AmazonBasics-Ventilated-...
5,B06XD7QSMS,In stock.,Computers & Accessories > Accessories & Periph...,AmazonBasics 4.0 Amp Dual USB Car Charger for ...,"1,311",4.2,499.00,https://www.amazon.in/AmazonBasics-Charger-App...
6,B01DN8TB5U,In stock.,"Office Products > Stationery > Notebooks, Writ...","AmazonBasics Classic Notebook, Plain - (130mm ...",807,4.0,299.00,https://www.amazon.in/AmazonBasics-Classic-Not...
7,B002VPE1QG,In stock.,Computers & Accessories > Accessories & Periph...,AmazonBasics Universal Travel Case for Small E...,"1,409",4.4,539.00,https://www.amazon.in/AmazonBasics-Universal-T...
8,B0106V1TYS,In stock.,Pet Supplies > Dogs > Litter & Housebreaking >...,AmazonBasics Jaw Clamp Scooper (26-inch),190,4.5,549.00,https://www.amazon.in/AmazonBasics-Jaw-Clamp-S...
9,B002VPE1WK,In stock.,Electronics > Cameras & Photography > Accessor...,AmazonBasics Backpack for SLR/DSLR Cameras and...,262,4.2,"1,999.00",https://www.amazon.in/AmazonBasics-Backpack-DS...


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('https://www.amazon.in/l/3848980031')
bs = BeautifulSoup(html, "html.parser")
titles = bs.find_all("div", class_="a-size-medium s-inline  s-access-title  a-text-normal")
print('List all the header tags :', *titles, sep='\n\n')

In [ ]:
from selenium import webdriver
import time

In [ ]:
extracted_data

In [ ]:
a=[]
driver = webdriver.Firefox(executable_path='C:\\Users\prasa\Downloads\geckodriver-v0.24.0-win64\geckodriver.exe')
#driver.get('https://www.amazon.in/l/3848980031')
driver.get('https://www.amazon.in/l/3848980031')

time.sleep(1)
htmlSource = driver.page_source
bs = BeautifulSoup(htmlSource, "html.parser")
titles = bs.find_all("a", class_="a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal")
for t in titles:
    a+=[t]  
    
    
print('List all the header tags :', *a, sep='\n\n')